### Imports

In [ ]:
# !pip install huggingface_hub transformers -q 

In [1]:
import os
from huggingface_hub import login

import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from sklearn.model_selection import train_test_split

### Colab

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

In [ ]:
# import zipfile

# zip_file_path1 = "/teamspace/studios/this_studio/Data.zip"
# zip_file_path2 = "/teamspace/studios/this_studio/Tokenizer.zip"
# zip_file_path3 = "/teamspace/studios/this_studio/Trained_model.zip"


# extract_dir = "/teamspace/studios/this_studio/"

# os.makedirs(extract_dir, exist_ok=True)

# with zipfile.ZipFile(zip_file_path1, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)
# with zipfile.ZipFile(zip_file_path2, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)
# with zipfile.ZipFile(zip_file_path3, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

### Data

In [ ]:
# with open("/Users/saileshpanda/Desktop/IITGN/NLP/Assignment3/archive/SST2-Data/SST2-Data/stanfordSentimentTreebank/stanfordSentimentTreebank/dictionary.txt", 'r') as f:
#     dictionary = f.read().split('\n')[:-1]
# with open("/Users/saileshpanda/Desktop/IITGN/NLP/Assignment3/archive/SST2-Data/SST2-Data/stanfordSentimentTreebank/stanfordSentimentTreebank/sentiment_labels.txt", 'r') as f:
#     labels = f.read().split('\n')[1:-1]

In [ ]:
# phrase_dict = {}
# for sent in dictionary:

#     sent, idx = sent.split('|')
#     phrase_dict[int(idx)] = sent

# label_dict = {}
# for sent in labels:

#     idx, score = sent.split('|')
#     score = float(score)
#     if score>=0 and score<=0.2:
#         lab = 0
#     elif score>0.2 and score<=0.4:
#         lab = 1
#     elif score>0.4 and score<=0.6:
#         lab = 2
#     elif score>0.6 and score<=0.8:
#         lab = 3
#     elif score>0.8 and score<=1.0:
#         lab = 4

#     label_dict[int(idx)] = lab


In [ ]:
# X = []
# y = []
# for i in range(len(label_dict)):

#     X.append(phrase_dict[i])
#     y.append(label_dict[i])

In [2]:
with open('/Users/saileshpanda/Desktop/IITGN/NLP/Assignment3/Data/X.txt', 'r') as f:
    X = f.read().split('\n')

y = torch.load('/Users/saileshpanda/Desktop/IITGN/NLP/Assignment3/Data/y.pt').to(torch.int)

/var/folders/jv/6j759gh52pjg5xgj136xc96w0000gn/T/ipykernel_37680/3641489098.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  y = torch.load('/Users/saileshpanda/Desktop/I

In [3]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3,
                                                    stratify=y, random_state=323)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2,
                                                    stratify=y_train_val, random_state=323)

### Model

In [ ]:
# login(token=os.environ['HUGGINGFACE_API_KEY'])

In [ ]:
# model_name = "meta-llama/Llama-3.2-1B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=5,
#     torch_dtype=torch.float16,
#     device_map="auto"

# )

# model.resize_token_embeddings(len(tokenizer))

# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token

# model.config.pad_token_id = tokenizer.pad_token_id

# model.save_pretrained("./Model")
# tokenizer.save_pretrained("./Tokenizer")

In [4]:
tokenizer = AutoTokenizer.from_pretrained("/Users/saileshpanda/Desktop/IITGN/NLP/Assignment3/Tokenizer")
model = AutoModelForSequenceClassification.from_pretrained("/Users/saileshpanda/Desktop/IITGN/NLP/Assignment3/Model")

### Preparing data for training

In [28]:
tokenized_X_train = tokenizer(X_train, truncation=True, padding=False)
tokenized_X_val = tokenizer(X_val, truncation=True, padding=False)
tokenized_X_test = tokenizer(X_test[:1000], truncation=True, padding=False)

In [29]:
class CustomDataset:
    def __init__(self, tokenized_inputs, labels):
        self.inputs = tokenized_inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.labels[idx],
        }

train_dataset = CustomDataset(tokenized_X_train, y_train)
test_dataset = CustomDataset(tokenized_X_test, y_test[:1000])
val_dataset = CustomDataset(tokenized_X_val, y_val)

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='pt')

### Training

In [ ]:
for name, param in model.named_parameters():
    if name != 'score.weight':
        param.requires_grad = False

In [ ]:
optim = AdamW([model.score.weight], lr=1e-5)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True, collate_fn=data_collator)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model.to(device)
for epoch in range(5):

    loss_train = np.array([])
    model.train()

    for idx, batch in enumerate(train_dataloader):

        optim.zero_grad()

        y_pred = model(batch.input_ids.to(device), batch.attention_mask.to(device))
        loss = torch.nn.functional.cross_entropy(y_pred.logits, batch.labels.type(torch.LongTensor).to(device))

        loss.backward()

        optim.step()

        loss_train = np.append(loss_train, loss.item())

    model.eval()
    loss_val = np.array([])
    with torch.no_grad():
      for batch in val_dataloader:
          y_pred = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
          loss = torch.nn.functional.cross_entropy(y_pred.logits, batch['labels'].type(torch.LongTensor)
          .to(device))
          loss_val = np.append(loss_val, loss.item())

    checkpoint_path = f"/teamspace/studios/this_studio/checkpoints/checkpoint_epoch_{epoch+1}"
    os.makedirs(checkpoint_path, exist_ok=True)
    model.save_pretrained(checkpoint_path)
    tokenizer.save_pretrained(checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")


    print(f"Epoch: {epoch} Train Loss: {loss_train.mean()} Evaluation loss: {loss_val.mean()}")

Testing

In [8]:
model_test = AutoModelForSequenceClassification.from_pretrained("/Users/saileshpanda/Desktop/IITGN/NLP/Assignment3/checkpoints_sst2/checkpoint_epoch_3")
tokenizer = AutoTokenizer.from_pretrained("/Users/saileshpanda/Desktop/IITGN/NLP/Assignment3/Tokenizer")

In [30]:
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, collate_fn=data_collator)

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
model_test.eval()

device = "cpu"

# Function to evaluate the model
def evaluate_model(model, dataloader, device):
    true_labels = []
    predictions = []
    
    with torch.no_grad():
        for batch in dataloader:
            inputs = {key: value.to(device) for key, value in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            
            true_labels.extend(labels.cpu().numpy())
            predictions.extend(preds.cpu().numpy())
    
    return true_labels, predictions

true_labels, predictions = evaluate_model(model_test, test_dataloader, device)

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='micro')
recall = recall_score(true_labels, predictions, average='micro')
f1 = f1_score(true_labels, predictions, average='micro')

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.5380
Precision: 0.5380
Recall: 0.5380
F1 Score: 0.5380
